# D-Hydro Boezemmodel V4 - Modeltests

Met deze notebook tonen we de prestaties van het D-Hydro model van Noorderzijlvest. 

Deze notebook is opgezet door Daniel Tollenaar (D2Hydro), Vincent de Looij (Waterschap Noorderzijlvest) en Siebe Bosch (HydroConsult).

We tonen hieronder de resultaten van een drietal tests:
1. Peil-in-rust: hoe goed het model leegloopt na initialisiatie met een hoge waterdiepte ten opzichte van het peil dat op de rekenpunten geldt
2. Half-maatgevende-afvoer: de afwijking ten opzichte van peil bij een stationaire half maatgevende afvoer. Half maatgevende afvoer is gedefinieerd als 7mm neerslag per dag, netto over het hele gebied
3. Dynamische afvoer: de afwijking ten opzichte van het geldende maaiveld bij een extreme dynamische bui.

Alle resultaten worden weergegeven in interactieve plots. Ook worden de resultaten weggeschreven als shape-files

In [1]:
import netCDF4 as nc
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, HoverTool, RangeSlider, CustomJS
from bokeh.layouts import grid

output_notebook(hide_banner=True)
tile_provider = get_provider(Vendors.CARTODBPOSITRON)

data_path = Path(r".\data").absolute().resolve()
beheerregister = data_path.joinpath("beheerregister")

invoerbestanden = {
    "peilgebieden": "Peilgebieden.shp"
}

modeltestbestanden = Path("modeltests")

## Peil In Rust - zomersituatie

Het peil in rust in de zomersituatie is berekend met een initiele waterdiepte van 6 meter, waarbij de eindwaterstanden per rekenknoop zijn vergeleken met het peil geldend voor de rekenknoop. De berekening is uitgevoerd voor een periode van 10 dagen.

In [2]:
ds = nc.Dataset(r".\modellen\boezemmodel\fm\DFM_OUTPUT_boezemmodel\boezemmodel_map.nc")
gpg_gdf = gpd.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]))
ref_level = "OPVAFWZP"

# inlezen waterhoogtes uit NetCDF
head_gdf = gpd.GeoDataFrame(
    data={"water_level": ds["mesh1d_s1"][-1],
          "geometry": [
              Point(coords) for coords in zip(ds["mesh1d_node_x"], ds["mesh1d_node_y"])
              ]}
    )

head_gdf.crs = "epsg:28992"
head_gdf["wl_diff"] = 0

# verschil met zomerpeil berekenen
for _, row in gpg_gdf.iterrows():
    zomer_peil = row[ref_level]
    set_idx = head_gdf.loc[
        head_gdf["geometry"].within(row["geometry"])
        ].index.to_list()
    if set_idx:
        head_gdf.loc[set_idx, ("zomer_peil")] = zomer_peil
        head_gdf.loc[set_idx, ("wl_diff")] = head_gdf.loc[
            set_idx]["water_level"
                     ] - zomer_peil

head_gdf.to_file(modeltestbestanden.joinpath("peil_in_rust.shp"))

In [6]:
p = figure()
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
gdf = head_gdf.to_crs("epsg:3857").copy()
gdf['x'] = gdf['geometry'].x
gdf['y'] = gdf['geometry'].y
df = gdf.drop('geometry', axis=1)

range_slider = RangeSlider(start=min(df["wl_diff"].min(), -0.5),
                           end=max(df["wl_diff"].max(), 0.5),
                           value=(-0.3,0.3),
                           step=.1,
                           title="Peil in rust - streefpeil [m]")

df["label"] = "tussen"
df["color"] = "white"
df.loc[df["wl_diff"] < range_slider.value[0], ["color", "label"]] = ["red", "lager"]
df.loc[df["wl_diff"] > range_slider.value[1], ["color", "label"]] = ["blue", "hoger"]
source = ColumnDataSource(df)

legend_callback = CustomJS(args=dict(source=source), code="""
    var nrows = source.get_length()
    var data = source.data;
    var f = cb_obj.value
    var diff = data['wl_diff']
    var color = data['color']
    var label = data['label']
    for (var i = 0; i < nrows; i++) {
        if (diff[i] < f[0]) {
          color[i] = "red"
          label[i] = "lager";
        } else if (diff[i] > f[1]) {
          color[i] = "blue"
          label[i] = "hoger";
        } else {
          color[i] = "white"
          label[i] = "tussen";
        }
    }
    source.change.emit();
""")

tooltips=[('peil:', '@zomer_peil{0.2f}'),
          ('berekend:', '@water_level{0.2f}'),
          ('verschil:', '@wl_diff{0.2f}')]

p.add_tile(tile_provider, name="background")
p.circle(source=source, fill_color='color', legend_field='label', line_alpha=0)
p.add_tools(HoverTool(tooltips=tooltips))

range_slider.js_on_change("value", legend_callback)

show(grid([range_slider,p]))